<a href="https://colab.research.google.com/github/atwine/Convo_Nets/blob/master/Colab_Xente_Fraud_Solution_Using_Deep_Neural_Nets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Xente Solution Using Deep Neural Nets
 By Atwine Mugume
 
I have been working on an online classification problem where we have to detect fraud.
Since I am now alittle conversant with some of these methods I want to try my luck at using them so that I can see if this will improve my score on the leader boards

In [0]:
#let me import some of the libraries that I will be using
import pandas as pd
import numpy as np

In [2]:
#Since this notebook is run in colab I have to import all my data from my laptop
from google.colab import files 
files.upload()

{}

In [3]:
#let's see if all our data is here
!ls

sample_data  test.csv  test_prc.csv  training.csv  train_prc.csv


In [0]:
#let's read the data into the notebook
train = pd.read_csv('training.csv')
test = pd.read_csv('test.csv')

In [5]:
#the data that I have imported above is not yet preprocessed
train.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0


In [6]:
#I have already preprocessed this data into two different csv files
#The only thing I need here is the id from the test data so that I can make a submission
test.head(4)

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy
0,TransactionId_50600,BatchId_35028,AccountId_2441,SubscriptionId_4426,CustomerId_2857,UGX,256,ProviderId_5,ProductId_3,airtime,ChannelId_3,1000.0,1000,2019-02-13T10:01:40Z,4
1,TransactionId_95109,BatchId_45139,AccountId_3439,SubscriptionId_2643,CustomerId_3874,UGX,256,ProviderId_5,ProductId_15,financial_services,ChannelId_3,2000.0,2000,2019-02-13T10:02:12Z,2
2,TransactionId_47357,BatchId_74887,AccountId_4841,SubscriptionId_3829,CustomerId_2857,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-50.0,50,2019-02-13T10:02:30Z,2
3,TransactionId_28185,BatchId_11025,AccountId_2685,SubscriptionId_4626,CustomerId_3105,UGX,256,ProviderId_5,ProductId_10,airtime,ChannelId_3,3000.0,3000,2019-02-13T10:02:38Z,4


In [7]:
#let me take out the transaction ID
TransactionID = test['TransactionId']
TransactionID[:5] #let's see the top 5 ids

0    TransactionId_50600
1    TransactionId_95109
2    TransactionId_47357
3    TransactionId_28185
4    TransactionId_22140
Name: TransactionId, dtype: object

### Pre-processed data

I am not going to bring in the pre-processed data on which I am going to run my neural net

In [0]:
train_prc = pd.read_csv('train_prc.csv')
test_prc = pd.read_csv('test_prc.csv')

In [9]:
train_prc.shape, test.shape

((95662, 92), (45019, 15))

In [10]:
train_prc.columns

Index(['Unnamed: 0', 'BatchId_0', 'BatchId_1', 'BatchId_2', 'BatchId_3',
       'BatchId_4', 'BatchId_5', 'BatchId_6', 'BatchId_7', 'BatchId_8',
       'BatchId_9', 'BatchId_10', 'BatchId_11', 'BatchId_12', 'BatchId_13',
       'BatchId_14', 'BatchId_15', 'BatchId_16', 'BatchId_17', 'AccountId_0',
       'AccountId_1', 'AccountId_2', 'AccountId_3', 'AccountId_4',
       'AccountId_5', 'AccountId_6', 'AccountId_7', 'AccountId_8',
       'AccountId_9', 'AccountId_10', 'AccountId_11', 'AccountId_12',
       'SubscriptionId_0', 'SubscriptionId_1', 'SubscriptionId_2',
       'SubscriptionId_3', 'SubscriptionId_4', 'SubscriptionId_5',
       'SubscriptionId_6', 'SubscriptionId_7', 'SubscriptionId_8',
       'SubscriptionId_9', 'SubscriptionId_10', 'SubscriptionId_11',
       'SubscriptionId_12', 'CustomerId_0', 'CustomerId_1', 'CustomerId_2',
       'CustomerId_3', 'CustomerId_4', 'CustomerId_5', 'CustomerId_6',
       'CustomerId_7', 'CustomerId_8', 'CustomerId_9', 'CustomerId_10',
       '

In [0]:
#let's take out the label of the data so that it doesn't get lost in there somewhere
label = train_prc['FraudResult']

#then we also drop this field from the data
train_prc = train_prc.drop(columns='FraudResult')

### Validation Set

In [0]:
#I need to keep a validation set of the data before we go so far
#I am going to keep the last 15000
x_val = train_prc.loc[80662:,]
y_val = label[80662:]

### Useable Data

In [0]:
train_df = train_prc.loc[:80661,]
label_ = label[:80662]

In [14]:
train_df.shape, label_.shape

((80662, 91), (80662,))

### Now let us split the data

We are going to use the sklearn model of splitting to plit the data and the we are going to work with the data imbalance before we beging to introduce our neural entwork.


In [0]:
from sklearn.model_selection import train_test_split

#let's split the data now
x_train, x_test, y_train, y_test = train_test_split(train_df,label_,test_size=0.2,random_state=2019)

In [16]:
#let's see the shapes
x_train.shape, y_train.shape,  x_test.shape, y_test.shape

((64529, 91), (64529,), (16133, 91), (16133,))

## Data Imbalance

Most times in fraud related datasets we normally have less fraud than there are fraud cases. We generally have data imbalance and would like to sort that out before we put the data into the  ML tuner.

In [17]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=27,ratio='minority')
x_train, y_train = smote.fit_sample(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [18]:
x_train.shape, y_train.shape

((128844, 91), (128844,))

In [20]:
print('After OverSampling, the shape of train_X: {}'.format(x_train.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train==0)))

After OverSampling, the shape of train_X: (128844, 91)
After OverSampling, the shape of train_y: (128844,) 

After OverSampling, counts of label '1': 64422
After OverSampling, counts of label '0': 64422


## Keras Model Building

Now that we have dealt with some of these things in the data its time to begin to deal with it by building a Deep Neural Network to Help solve this problem.

I am going to be using Google Colab because they have a free GPU and TPU


In [21]:
#let's import the necessary libraries to be able to use the DNN
from keras import Sequential
from keras.layers import Dense

Using TensorFlow backend.


## First Model Attempt

In [25]:
#Design the deep neural network [Small + 1 layer]
model  = Sequential()
model.add(Dense(256,input_dim = x_train.shape[1],activation="relu"))
model.add(Dense(256,activation="relu"))
model.add(Dense(1,activation = "sigmoid")) #activation = sigmoid for binary classification

model.compile(optimizer = "Adam",loss="binary_crossentropy",metrics=["accuracy"])

model.fit(x_train,y_train, validation_data = (x_val,y_val),epochs=10, batch_size=64)

Train on 128844 samples, validate on 15000 samples
Epoch 1/10
128844/128844 [==============================] - 7s 56us/step - loss: 8.0590 - acc: 0.5000 - val_loss: 0.0570 - val_acc: 0.9965
Epoch 2/10
128844/128844 [==============================] - 7s 54us/step - loss: 8.0590 - acc: 0.5000 - val_loss: 0.0570 - val_acc: 0.9965
Epoch 3/10
128844/128844 [==============================] - 7s 55us/step - loss: 8.0590 - acc: 0.5000 - val_loss: 0.0570 - val_acc: 0.9965
Epoch 4/10
128844/128844 [==============================] - 7s 54us/step - loss: 8.0590 - acc: 0.5000 - val_loss: 0.0570 - val_acc: 0.9965
Epoch 5/10
128844/128844 [==============================] - 7s 54us/step - loss: 8.0590 - acc: 0.5000 - val_loss: 0.0570 - val_acc: 0.9965
Epoch 6/10
128844/128844 [==============================] - 7s 54us/step - loss: 8.0590 - acc: 0.5000 - val_loss: 0.0570 - val_acc: 0.9965
Epoch 7/10
128844/128844 [==============================] - 7s 54us/step - loss: 8.0590 - acc: 0.5000 - val_loss: 0

In [26]:
#let's see the results
results = model.evaluate(x_val, y_val)

15000/15000 [==============================] - 0s 26us/step


In [27]:
print('Test accuracy: ', results[1])

Test accuracy:  0.9964666666666666


In [28]:
pred = model.predict(test_prc)
len(pred)

45019

In [0]:
#let's make a sample submission
data = pd.DataFrame({
 'TransactionId':TransactionID,
 'FraudResult':pred[:,0]   
})

In [30]:
#data.set_index('TransactionId',inplace=True)
data[data['FraudResult']==1] #I am not getting any scores on the fraud cases so this model can't be true

,TransactionId,FraudResult


## Second Attempt 

This time we want to scale the data (missed to do that before)

In [0]:
from sklearn.preprocessing  import StandardScaler

#Create a standardscaler instance and fit the data
scaler = StandardScaler()
x_train_sc = scaler.fit_transform(x_train)
x_test_sc = scaler.fit_transform(x_test)
x_val_sc = scaler.fit_transform(x_val)

In [33]:
model  = Sequential()
model.add(Dense(512,input_dim = x_train_sc.shape[1],activation="relu"))
model.add(Dense(1,activation = "relu"))

model.compile(optimizer = "Adam",loss="binary_crossentropy",metrics=["accuracy"])

model.fit(x_train_sc,y_train, validation_data = (x_val_sc,y_val),epochs=3, batch_size=64)

Train on 128844 samples, validate on 15000 samples
Epoch 1/3
128844/128844 [==============================] - 5s 40us/step - loss: 0.2416 - acc: 0.6151 - val_loss: 0.6816 - val_acc: 0.9241
Epoch 2/3
128844/128844 [==============================] - 5s 37us/step - loss: 0.0152 - acc: 0.6167 - val_loss: 0.8592 - val_acc: 0.9163
Epoch 3/3
128844/128844 [==============================] - 5s 38us/step - loss: 0.0189 - acc: 0.5529 - val_loss: 0.4386 - val_acc: 0.9532


In [34]:
results = model.evaluate(x_val, y_val)

15000/15000 [==============================] - 0s 27us/step


In [35]:
print('Test accuracy: ', results[1])

Test accuracy:  0.0


## Third Attempt

Let's add two medium dense layers to see if our predictions become better.

In [107]:
model  = Sequential()
model.add(Dense(512,input_dim = x_train_sc.shape[1],activation="relu"))
model.add(Dense(512,activation="relu"))
model.add(Dense(1,activation = "sigmoid"))

model.compile(optimizer = "Adam",loss="binary_crossentropy",metrics=["accuracy"])

model.fit(x_train_sc,y_train_subsample, validation_data = (x_val_sc,y_val),epochs=3, batch_size=64)

Train on 214 samples, validate on 15000 samples
Epoch 1/3
214/214 [==============================] - 2s 8ms/step - loss: 0.5972 - acc: 0.7150 - val_loss: 0.6502 - val_acc: 0.6416
Epoch 2/3
214/214 [==============================] - 0s 2ms/step - loss: 0.2771 - acc: 0.9065 - val_loss: 0.9070 - val_acc: 0.5734
Epoch 3/3
214/214 [==============================] - 0s 2ms/step - loss: 0.1632 - acc: 0.9439 - val_loss: 0.9722 - val_acc: 0.6383


In [108]:
results = model.evaluate(x_test_sc, y_test)

16133/16133 [==============================] - 1s 53us/step


In [109]:
print('Test accuracy: ', results[1])

Test accuracy:  0.6561085972887625


## Fourth Attempt

Let's design two large layers

In [116]:
model  = Sequential()
model.add(Dense(1024,input_dim = x_train_sc.shape[1],activation="relu"))
model.add(Dense(1024,activation="relu"))
model.add(Dense(1,activation = "sigmoid"))

model.compile(optimizer = "Adam",loss="binary_crossentropy",metrics=["accuracy"])

model.fit(x_train_sc,y_train_subsample, validation_data = (x_val_sc,y_val),epochs=3, batch_size=64)

Train on 214 samples, validate on 15000 samples
Epoch 1/3
214/214 [==============================] - 3s 13ms/step - loss: 0.5234 - acc: 0.7290 - val_loss: 1.1614 - val_acc: 0.4638
Epoch 2/3
214/214 [==============================] - 1s 5ms/step - loss: 0.2197 - acc: 0.9112 - val_loss: 0.8794 - val_acc: 0.7293
Epoch 3/3
214/214 [==============================] - 1s 5ms/step - loss: 0.1067 - acc: 0.9673 - val_loss: 1.1494 - val_acc: 0.6808


In [117]:
results = model.evaluate(x_test_sc, y_test)


16133/16133 [==============================] - 2s 118us/step


In [118]:
print('Test accuracy: ', results[1])
#The accuracy increased when we increased the layers

Test accuracy:  0.6471827930329139


## Fifth Attempt

I am going to add 4 large layers and run them 20 epochs

In [37]:
model  = Sequential()
model.add(Dense(1024,input_dim = x_train_sc.shape[1],activation="relu"))
model.add(Dense(102,activation="relu"))
model.add(Dense(1024,activation="relu"))
model.add(Dense(1,activation = "sigmoid"))

model.compile(optimizer = "Adam",loss="binary_crossentropy",metrics=["accuracy"])

model.fit(x_train_sc,y_train, validation_data = (x_val_sc,y_val),epochs=50, batch_size=64)

Train on 128844 samples, validate on 15000 samples
Epoch 1/50
128844/128844 [==============================] - 25s 191us/step - loss: 0.0101 - acc: 0.9972 - val_loss: 0.1466 - val_acc: 0.9629
Epoch 2/50
128844/128844 [==============================] - 22s 174us/step - loss: 0.0039 - acc: 0.9993 - val_loss: 0.2227 - val_acc: 0.9476
Epoch 3/50
128844/128844 [==============================] - 23s 181us/step - loss: 0.0030 - acc: 0.9995 - val_loss: 0.1086 - val_acc: 0.9743
Epoch 4/50
128844/128844 [==============================] - 22s 173us/step - loss: 0.0028 - acc: 0.9995 - val_loss: 0.1562 - val_acc: 0.9721
Epoch 5/50
128844/128844 [==============================] - 24s 185us/step - loss: 0.0021 - acc: 0.9996 - val_loss: 0.2761 - val_acc: 0.9549
Epoch 6/50
128844/128844 [==============================] - 23s 181us/step - loss: 0.0025 - acc: 0.9996 - val_loss: 0.2349 - val_acc: 0.9666
Epoch 7/50
128844/128844 [==============================] - 21s 160us/step - loss: 0.0020 - acc: 0.9997

In [38]:
results = model.evaluate(x_test_sc, y_test)

16133/16133 [==============================] - 1s 49us/step


In [39]:
print('Test accuracy: ', results[1])

Test accuracy:  0.9514659393789128
